In [1]:
print("="*80)
print("STEP 1: IMPORTING LIBRARIES")
print("="*80)

# Deep Learning Framework
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.optimizers import Adam

# Data Processing
import numpy as np
import pickle
import re
import requests

# Data Splitting
from sklearn.model_selection import train_test_split

# Utilities
import time
import sys

print("✓ All libraries imported successfully")
print()

STEP 1: IMPORTING LIBRARIES
✓ All libraries imported successfully



In [2]:
print("="*80)
print("STEP 2: SETTING RANDOM SEEDS")
print("="*80)

# This ensures you get the same results every time you run the code
np.random.seed(42)
tf.random.set_seed(42)

print("✓ Random seeds set to 42")
print("  This ensures reproducible results")
print()

STEP 2: SETTING RANDOM SEEDS
✓ Random seeds set to 42
  This ensures reproducible results



In [3]:
print("="*80)
print("STEP 3: DOWNLOADING SHAKESPEARE DATASET")
print("="*80)

def download_text_data():
    """
    Downloads Shakespeare's complete works from Project Gutenberg.
    This is our training data source.
    """
    url = "https://www.gutenberg.org/files/100/100-0.txt"

    try:
        print(f"Downloading from: {url}")
        response = requests.get(url, timeout=30)
        response.raise_for_status()
        text = response.text
        print(f"✓ Downloaded {len(text):,} characters")
        return text
    except Exception as e:
        print(f"✗ Download failed: {e}")
        print("Using sample text instead...")
        # Fallback sample text (repeated for sufficient data)
        sample = """To be, or not to be, that is the question:
Whether 'tis nobler in the mind to suffer
The slings and arrows of outrageous fortune,
Or to take arms against a sea of troubles
And by opposing end them. To die—to sleep,
No more; and by a sleep to say we end
The heart-ache and the thousand natural shocks
That flesh is heir to: 'tis a consummation
Devoutly to be wish'd."""
        return sample * 500  # Repeat to have enough data

# Download the text
raw_text = download_text_data()
print()


STEP 3: DOWNLOADING SHAKESPEARE DATASET
✓ Downloaded 5,359,444 characters



In [4]:
print("="*80)
print("STEP 4: PREPROCESSING TEXT DATA")
print("="*80)

def preprocess_text(text):
    """
    Cleans and normalizes the text for training.

    Steps:
    1. Convert to lowercase
    2. Remove special characters (keep only a-z, spaces, basic punctuation)
    3. Normalize whitespace
    """
    print("Processing steps:")

    # Step 1: Lowercase
    text = text.lower()
    print("  1. Converted to lowercase")

    # Step 2: Remove unwanted characters
    text = re.sub(r'[^a-z\s.,!?;:\'-]', '', text)
    print("  2. Removed special characters")

    # Step 3: Normalize whitespace
    text = re.sub(r'\s+', ' ', text).strip()
    print("  3. Normalized whitespace")

    return text

# Preprocess the text
text = preprocess_text(raw_text)
print(f"✓ Final text length: {len(text):,} characters")

# Use a subset for optimal speed/accuracy balance
text = text[:200000]  # 200K characters
print(f"✓ Using {len(text):,} characters for training")
print()


STEP 4: PREPROCESSING TEXT DATA
Processing steps:
  1. Converted to lowercase
  2. Removed special characters
  3. Normalized whitespace
✓ Final text length: 5,245,169 characters
✓ Using 200,000 characters for training



In [5]:
print("="*80)
print("STEP 5: CREATING CHARACTER VOCABULARY")
print("="*80)

# Get unique characters
chars = sorted(list(set(text)))
vocab_size = len(chars)

print(f"✓ Found {vocab_size} unique characters")
print(f"Characters: {chars}")

# Create mappings: character <-> index
char_to_idx = {ch: i for i, ch in enumerate(chars)}
idx_to_char = {i: ch for i, ch in enumerate(chars)}

print(f"✓ Created character-to-index mappings")
print(f"  Example: '{chars[0]}' -> {char_to_idx[chars[0]]}")
print(f"  Example: {0} -> '{idx_to_char[0]}'")
print()

STEP 5: CREATING CHARACTER VOCABULARY
✓ Found 34 unique characters
Characters: [' ', '!', ',', '-', '.', ':', ';', '?', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
✓ Created character-to-index mappings
  Example: ' ' -> 0
  Example: 0 -> ' '



In [6]:
print("="*80)
print("STEP 6: CREATING TRAINING SEQUENCES")
print("="*80)

# Hyperparameters for sequence creation
SEQUENCE_LENGTH = 40  # How many characters to use as input
STEP_SIZE = 3         # How many characters to skip between sequences

print(f"Configuration:")
print(f"  Sequence Length: {SEQUENCE_LENGTH} characters")
print(f"  Step Size: {STEP_SIZE} characters")
print()

# Create sequences
sequences = []      # Will store input sequences
next_chars = []     # Will store the target (next character)

print("Creating sequences...")
for i in range(0, len(text) - SEQUENCE_LENGTH, STEP_SIZE):
    # Input: 40 characters
    sequences.append(text[i:i + SEQUENCE_LENGTH])
    # Output: The next character after those 40
    next_chars.append(text[i + SEQUENCE_LENGTH])

n_sequences = len(sequences)
print(f"✓ Created {n_sequences:,} sequences")
print()

# Example of a sequence:
print("Example sequence:")
print(f"  Input:  '{sequences[0]}'")
print(f"  Target: '{next_chars[0]}'")
print()


STEP 6: CREATING TRAINING SEQUENCES
Configuration:
  Sequence Length: 40 characters
  Step Size: 3 characters

Creating sequences...
✓ Created 66,654 sequences

Example sequence:
  Input:  'start of the project gutenberg ebook the'
  Target: ' '



In [7]:
print("="*80)
print("STEP 7: ENCODING SEQUENCES TO NUMBERS")
print("="*80)

print("Creating numerical arrays...")

# Initialize arrays
# X: Input sequences (numbers instead of characters)
X = np.zeros((n_sequences, SEQUENCE_LENGTH), dtype=np.int32)

# y: Output (one-hot encoded)
y = np.zeros((n_sequences, vocab_size), dtype=np.int32)

# Fill the arrays
print("Encoding sequences (this may take a moment)...")
for i, sequence in enumerate(sequences):
    # Encode each character in the sequence
    for t, char in enumerate(sequence):
        X[i, t] = char_to_idx[char]

    # One-hot encode the target character
    y[i, char_to_idx[next_chars[i]]] = 1

    # Progress indicator
    if (i + 1) % 10000 == 0:
        print(f"  Encoded {i+1:,}/{n_sequences:,} sequences...")

print(f"✓ X shape: {X.shape} (sequences, sequence_length)")
print(f"✓ y shape: {y.shape} (sequences, vocabulary_size)")
print()

STEP 7: ENCODING SEQUENCES TO NUMBERS
Creating numerical arrays...
Encoding sequences (this may take a moment)...
  Encoded 10,000/66,654 sequences...
  Encoded 20,000/66,654 sequences...
  Encoded 30,000/66,654 sequences...
  Encoded 40,000/66,654 sequences...
  Encoded 50,000/66,654 sequences...
  Encoded 60,000/66,654 sequences...
✓ X shape: (66654, 40) (sequences, sequence_length)
✓ y shape: (66654, 34) (sequences, vocabulary_size)



In [8]:
print("="*80)
print("STEP 8: SPLITTING DATA (80% TRAIN, 20% VALIDATION)")
print("="*80)

X_train, X_val, y_train, y_val = train_test_split(
    X, y,
    test_size=0.2,      # 20% for validation
    random_state=42,    # Reproducible split
    shuffle=True        # Shuffle before splitting
)

print(f"✓ Training set: {len(X_train):,} sequences")
print(f"✓ Validation set: {len(X_val):,} sequences")
print()

STEP 8: SPLITTING DATA (80% TRAIN, 20% VALIDATION)
✓ Training set: 53,323 sequences
✓ Validation set: 13,331 sequences



In [9]:
print("="*80)
print("STEP 9: BUILDING LSTM MODEL ARCHITECTURE")
print("="*80)

print("Creating model layers:")

model = Sequential([
    # Layer 1: Embedding
    # Converts character indices to dense vectors
    Embedding(
        input_dim=vocab_size,           # Number of unique characters
        output_dim=128,                 # Dimension of embedding vectors
        input_length=SEQUENCE_LENGTH    # Length of input sequences
    ),

    # Layer 2: First LSTM
    # Learns patterns in sequences, returns sequences for next layer
    LSTM(
        256,                      # Number of LSTM units
        return_sequences=True,    # Return full sequence
        dropout=0.2,             # Dropout for regularization
        recurrent_dropout=0.2    # Dropout on recurrent connections
    ),

    # Layer 3: Second LSTM
    # Further pattern learning
    LSTM(
        256,                      # Number of LSTM units
        dropout=0.2,
        recurrent_dropout=0.2
    ),

    # Layer 4: Dropout
    # Prevent overfitting
    Dropout(0.3),

    # Layer 5: Dense hidden layer
    # Non-linear transformation
    Dense(128, activation='relu'),

    # Layer 6: Dropout
    Dropout(0.2),

    # Layer 7: Output layer
    # Predicts probability distribution over all characters
    Dense(vocab_size, activation='softmax')
])

print("✓ Model architecture created")
print()

# Display model summary
print("Model Summary:")
print("-" * 80)
model.summary()
print("-" * 80)
print()


STEP 9: BUILDING LSTM MODEL ARCHITECTURE
Creating model layers:


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


✓ Model architecture created

Model Summary:
--------------------------------------------------------------------------------


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

--------------------------------------------------------------------------------



In [10]:
print("="*80)
print("STEP 10: COMPILING THE MODEL")
print("="*80)

# Create optimizer
optimizer = Adam(learning_rate=0.001)

# Compile the model
model.compile(
    loss='categorical_crossentropy',  # Loss function for classification
    optimizer=optimizer,               # Optimization algorithm
    metrics=['accuracy']               # Metric to monitor
)

print("✓ Model compiled with:")
print("  - Optimizer: Adam (learning_rate=0.001)")
print("  - Loss: Categorical Crossentropy")
print("  - Metrics: Accuracy")
print()

STEP 10: COMPILING THE MODEL
✓ Model compiled with:
  - Optimizer: Adam (learning_rate=0.001)
  - Loss: Categorical Crossentropy
  - Metrics: Accuracy



In [11]:
print("="*80)
print("STEP 11: CONFIGURING TRAINING CALLBACKS")
print("="*80)

callbacks = [
    # Early Stopping: Stop training if validation accuracy doesn't improve
    EarlyStopping(
        monitor='val_accuracy',        # Metric to monitor
        patience=5,                    # Wait 5 epochs before stopping
        restore_best_weights=True,     # Restore best weights
        verbose=1
    ),

    # Model Checkpoint: Save the best model during training
    ModelCheckpoint(
        'best_model.keras',            # File to save model
        monitor='val_accuracy',        # Metric to monitor
        save_best_only=True,           # Only save when improving
        verbose=1
    )
]

print("✓ Callbacks configured:")
print("  1. Early Stopping (patience=5)")
print("  2. Model Checkpoint (saves best model)")
print()

STEP 11: CONFIGURING TRAINING CALLBACKS
✓ Callbacks configured:
  1. Early Stopping (patience=5)
  2. Model Checkpoint (saves best model)



In [14]:
print("="*80)
print("STEP 12: TRAINING THE MODEL")
print("="*80)

# Training parameters
BATCH_SIZE = 256  # Number of samples per batch
EPOCHS = 100       # Maximum number of epochs

print(f"Training configuration:")
print(f"  Batch Size: {BATCH_SIZE}")
print(f"  Max Epochs: {EPOCHS}")
print(f"  Total Training Samples: {len(X_train):,}")
print(f"  Steps per Epoch: {len(X_train) // BATCH_SIZE}")
print()

print("Starting training...")
print("=" * 80)

# Start timer
start_time = time.time()

# Train the model
history = model.fit(
    X_train, y_train,
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    validation_data=(X_val, y_val),
    callbacks=callbacks,
    verbose=1  # Show progress bar
)

# Calculate training time
training_time = time.time() - start_time

print("=" * 80)
print(f"✓ Training completed in {training_time/60:.2f} minutes")
print()


STEP 12: TRAINING THE MODEL
Training configuration:
  Batch Size: 256
  Max Epochs: 100
  Total Training Samples: 53,323
  Steps per Epoch: 208

Starting training...
Epoch 1/100
209/209 ━━━━━━━━━━━━━━━━━━━━ 0s 261ms/step - accuracy: 0.3185 - loss: 2.3189
Epoch 1: val_accuracy improved from 0.28738 to 0.36126, saving model to best_model.keras
209/209 ━━━━━━━━━━━━━━━━━━━━ 57s 273ms/step - accuracy: 0.3186 - loss: 2.3186 - val_accuracy: 0.3613 - val_loss: 2.1344
Epoch 2/100
209/209 ━━━━━━━━━━━━━━━━━━━━ 0s 258ms/step - accuracy: 0.3630 - loss: 2.1531
Epoch 2: val_accuracy improved from 0.36126 to 0.39359, saving model to best_model.keras
209/209 ━━━━━━━━━━━━━━━━━━━━ 57s 273ms/step - accuracy: 0.3630 - loss: 2.1529 - val_accuracy: 0.3936 - val_loss: 2.0154
Epoch 3/100
209/209 ━━━━━━━━━━━━━━━━━━━━ 0s 256ms/step - accuracy: 0.3912 - loss: 2.0400
Epoch 3: val_accuracy improved from 0.39359 to 0.41925, saving model to best_model.keras
209/209 ━━━━━━━━━━━━━━━━━━━━ 81s 270ms/step - accuracy: 0.39

In [16]:
print("="*80)
print("STEP 13: EVALUATING MODEL PERFORMANCE")
print("="*80)

# Evaluate on training set
print("Evaluating on training set...")
train_loss, train_accuracy = model.evaluate(X_train, y_train, verbose=0)

print(f"Training Set Performance:")
print(f"  Loss: {train_loss:.4f}")
print(f"  Accuracy: {train_accuracy*100:.2f}%")
print()

# Evaluate on validation set
print("Evaluating on validation set...")
val_loss, val_accuracy = model.evaluate(X_val, y_val, verbose=0)

print(f"Validation Set Performance:")
print(f"  Loss: {val_loss:.4f}")
print(f"  Accuracy: {val_accuracy*100:.2f}%")
print()

# Check if requirements are met
print("="*80)
print("REQUIREMENTS CHECK")
print("="*80)

train_pass = train_accuracy >= 0.90
val_pass = val_accuracy >= 0.90
time_pass = training_time <= 3600  # 60 minutes

print(f"Training Accuracy >= 90%:   {'✓ PASS' if train_pass else '✗ FAIL'}")
print(f"Validation Accuracy >= 90%: {'✓ PASS' if val_pass else '✗ FAIL'}")
print(f"Training Time < 60 min:     {'✓ PASS' if time_pass else '✗ FAIL'}")

if train_pass and val_pass and time_pass:
    print("\n🎉 SUCCESS! All requirements met!")
else:
    print("\n⚠ Some requirements not met. See suggestions above.")
print()

STEP 13: EVALUATING MODEL PERFORMANCE
Evaluating on training set...
Training Set Performance:
  Loss: 0.9441
  Accuracy: 69.05%

Evaluating on validation set...
Validation Set Performance:
  Loss: 1.6533
  Accuracy: 52.73%

REQUIREMENTS CHECK
Training Accuracy >= 90%:   ✗ FAIL
Validation Accuracy >= 90%: ✗ FAIL
Training Time < 60 min:     ✓ PASS

⚠ Some requirements not met. See suggestions above.



In [17]:
print("="*80)
print("STEP 14: CREATING TEXT GENERATION FUNCTION")
print("="*80)

def generate_text(model, seed_text, length=200, temperature=0.7):
    """
    Generate text using the trained model.

    Args:
        model: Trained LSTM model
        seed_text: Starting text (will be padded/truncated to SEQUENCE_LENGTH)
        length: Number of characters to generate
        temperature: Controls randomness (0.1=conservative, 2.0=creative)

    Returns:
        Generated text string
    """
    # Preprocess seed text
    seed_text = seed_text.lower()
    seed_text = re.sub(r'[^a-z\s.,!?;:\'-]', '', seed_text)

    # Ensure seed is right length
    if len(seed_text) < SEQUENCE_LENGTH:
        seed_text = seed_text + ' ' * (SEQUENCE_LENGTH - len(seed_text))
    seed_text = seed_text[-SEQUENCE_LENGTH:]

    generated_text = seed_text

    # Generate character by character
    for i in range(length):
        # Encode current sequence
        x_pred = np.zeros((1, SEQUENCE_LENGTH), dtype=np.int32)
        for t, char in enumerate(seed_text):
            if char in char_to_idx:
                x_pred[0, t] = char_to_idx[char]

        # Predict next character probabilities
        predictions = model.predict(x_pred, verbose=0)[0]

        # Apply temperature
        predictions = np.log(predictions + 1e-7) / temperature
        exp_preds = np.exp(predictions)
        predictions = exp_preds / np.sum(exp_preds)

        # Sample next character
        next_idx = np.random.choice(len(predictions), p=predictions)
        next_char = idx_to_char[next_idx]

        # Update
        generated_text += next_char
        seed_text = seed_text[1:] + next_char

    return generated_text

print("✓ Text generation function created")
print()


STEP 14: CREATING TEXT GENERATION FUNCTION
✓ Text generation function created



In [18]:
print("="*80)
print("STEP 15: GENERATING SAMPLE TEXTS")
print("="*80)

# Test with different seeds and temperatures
test_seeds = [
    "to be or not to be",
    "shall i compare thee to",
    "what light through yonder"
]

temperatures = [0.5, 0.7, 1.0]

for seed in test_seeds:
    print("\n" + "="*80)
    print(f"SEED: '{seed}'")
    print("="*80)

    for temp in temperatures:
        print(f"\nTemperature: {temp}")
        print("-"*80)
        generated = generate_text(model, seed, length=200, temperature=temp)
        print(generated)

print()

STEP 15: GENERATING SAMPLE TEXTS

SEED: 'to be or not to be'

Temperature: 0.5
--------------------------------------------------------------------------------
to be or not to be                      oure far thy dead, and love that would make that hours than the fill will receive i can the story and her not care, the sworn to make the love be than i must come, and all the sound with the world wor

Temperature: 0.7
--------------------------------------------------------------------------------
to be or not to be                      ore. all the duke of parts are of friends; i will know not betender, and wherefore all the breast palace. king. aside. helena. the saffet make me the love of world with a balace that with thy home. so

Temperature: 1.0
--------------------------------------------------------------------------------
to be or not to be                      at thy ard, which rroper not before, to by itsulf to the jack-ons freserving own, my sake. o first to from this vidd, wh

In [15]:
print("="*80)
print("STEP 16: SAVING MODEL AND METADATA")
print("="*80)

# Save the final model
model.save('lstm_text_generator.keras')
print("✓ Model saved as 'lstm_text_generator.keras'")

# Save metadata (vocabularies and configuration)
metadata = {
    'char_to_idx': char_to_idx,
    'idx_to_char': idx_to_char,
    'vocab_size': vocab_size,
    'sequence_length': SEQUENCE_LENGTH,
    'chars': chars
}

with open('model_metadata.pkl', 'wb') as f:
    pickle.dump(metadata, f)
print("✓ Metadata saved as 'model_metadata.pkl'")

# Save training history
with open('training_history.pkl', 'wb') as f:
    pickle.dump(history.history, f)
print("✓ Training history saved as 'training_history.pkl'")

print()


STEP 16: SAVING MODEL AND METADATA
✓ Model saved as 'lstm_text_generator.keras'
✓ Metadata saved as 'model_metadata.pkl'
✓ Training history saved as 'training_history.pkl'

